In [0]:
%run ../Includes/Copy-Datasets

In [0]:
%sql
CREATE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`

In [0]:
%sql
SELECT * FROM orders

In [0]:
%sql
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`

In [0]:
%sql
DESCRIBE HISTORY orders

In [0]:
%sql
INSERT OVERWRITE orders
SELECT * FROM parquet.`${dataset.bookstore}/orders`

In [0]:
%sql
DESCRIBE HISTORY orders

In [0]:
%sql
INSERT OVERWRITE orders
SELECT *, current_timestamp() FROM parquet.`${dataset.bookstore}/orders`

In [0]:
%sql
INSERT INTO orders
SELECT * FROM parquet.`${dataset.bookstore}/orders-new`

In [0]:
%sql
SELECT count(*) FROM orders

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW customers_updates AS 
SELECT * FROM json.`${dataset.bookstore}/customers-json-new`;

MERGE INTO customers c
USING customers_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET email = u.email, updated = u.updated
WHEN NOT MATCHED THEN INSERT *

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW books_updates
   (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING CSV
OPTIONS (
  path = "${dataset.bookstore}/books-csv-new",
  header = "true",
  delimiter = ";"
);

SELECT * FROM books_updates

In [0]:
%sql
MERGE INTO books b
USING books_updates u
ON b.book_id = u.book_id AND b.title = u.title
WHEN NOT MATCHED AND u.category = 'Computer Science' THEN 
  INSERT *